In [1]:
# Imports
# Packages for numerics + dataframes
import pandas as pd
import numpy as np

# Packages for visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Packages for date conversions for calculating trip durations
from datetime import datetime
from datetime import date
from datetime import timedelta

# Packages for OLS, MLR, confusion matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics # For confusion matrix
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
file_path = '/content/drive/My Drive/Advanced Data Analytics Certificate/Activity Datasets/2017_Yellow_Taxi_Trip_Data.csv'
df0=pd.read_csv(file_path)

In [4]:
df = df0.copy()

In [5]:
# Remove the target column from the features
X = df.drop(columns=['fare_amount'])

# Set y variable
y = df[['fare_amount']]

# BONUS CONTENT

More work must be done to prepare the predictions to be used as inputs into the model for the upcoming course. This work will be broken into the following steps:

1. Get the model's predictions on the full dataset.

2. Impute the constant fare rate of \$52 for all trips with rate codes of `2`.

3. Check the model's performance on the full dataset.

4. Save the final predictions and `mean_duration` and `mean_distance` columns for downstream use.




### 1. Predict on full dataset

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [7]:
X_test.columns

Index(['Unnamed: 0', 'VendorID', 'tpep_pickup_datetime',
       'tpep_dropoff_datetime', 'passenger_count', 'trip_distance',
       'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID',
       'payment_type', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount'],
      dtype='object')

Since we are pretending X_test and y_test are future use-cases, where a user is using the app to call a taxi and the app will provide an estimate of fare amount, after the user input some necessary infomation, such as passenger count, and desination, some of the columns in X_test need to be removed because they are unknown at the time when the app is used. For example, 'trip_distance' and 'tpep_dropoff_datetime' need to be removed.

Assumption 1: We can use 'tpep_pickup_datetime' as the time when the app is used to book the trip. This time can be used to determine if it is in rush hour.

In [8]:
X_test = X_test.drop(['Unnamed: 0', 'tpep_dropoff_datetime', 'trip_distance',
               'store_and_fwd_flag',
               'payment_type', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
               'total_amount'
               ], axis=1)
X_test.columns

Index(['VendorID', 'tpep_pickup_datetime', 'passenger_count', 'RatecodeID',
       'PULocationID', 'DOLocationID'],
      dtype='object')

Assumption 2: Since X_test is used to simulate the actual use-cases, only the following columns are deemed to be available.
* 'VendorID',
* 'tpep_pickup_datetime',
* 'passenger_count',
* 'RatecodeID',
* 'PULocationID',
* 'DOLocationID'

Assumption 3: (Time line)

At this moment, we known X_train and y_train. We will know X_test after we build the model.

Note: model include lr-model, all assumptions, all necessary tables, all parameters, and rules used in the prediction process.

We will not know y_test until the future trips are done. This means that we will know y_test after we computed y_test_pred.

We can compute y_train_pred in the same way for computing y_test_pred.


The next step here is to separate out the rows in X_train and y_train where RatecodeID == 2.

Rule 1:

In order to do this, first we will combine X_train and y_train into a single dataframe. We call this dataframe Xy_train. Then we need to separate the rows where RatecodeID == 2 and call this dataframe Xy_train_2. Call the remaining dataframe Xy_train_1.

In [9]:
# prompt: combine X_train and y_train into a single dataframe and call this dataframe Xy_train.
# Then separate the rows where RatecodeID == 2 and call this dataframe Xy_train_2.
# Call the remaining dataframe Xy_train_1.

# Combine X_train and y_train
Xy_train = pd.concat([X_train, y_train], axis=1)

# Separate rows where RatecodeID == 2
Xy_train_2 = Xy_train[Xy_train['RatecodeID'] == 2]
Xy_train_1 = Xy_train[Xy_train['RatecodeID'] != 2]


In [10]:
# prompt: reset index for Xy_train, Xy_train_1, and Xy_train_2

# Reset index for Xy_train, Xy_train_1, and Xy_train_2
Xy_train = Xy_train.reset_index(drop=True)
Xy_train_1 = Xy_train_1.reset_index(drop=True)
Xy_train_2 = Xy_train_2.reset_index(drop=True)


In [11]:
Xy_train.head(3)

,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,fare_amount
0,31150737,2,04/16/2017 11:08:31 PM,04/16/2017 11:24:44 PM,1,6.02,1,N,229,168,2,0.5,0.5,0.0,0.0,0.3,20.8,19.5
1,33778519,2,04/24/2017 6:52:50 AM,04/24/2017 6:55:51 AM,1,0.66,1,N,249,114,1,0.0,0.5,1.0,0.0,0.3,6.3,4.5
2,43955786,1,05/21/2017 2:36:49 AM,05/21/2017 2:42:24 AM,2,1.40,1,N,114,231,1,0.5,0.5,1.0,0.0,0.3,8.8,6.5


In [12]:
display(Xy_train.shape)
display(Xy_train_2.shape)
display(Xy_train_1.shape)
display(Xy_train_2.shape[0] + Xy_train_1.shape[0])


(18159, 18)

(416, 18)

(17743, 18)

18159

In [13]:
Xy_train_2.head(3)

,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,fare_amount
0,92435973,2,11/01/2017 9:30:09 PM,11/01/2017 9:30:16 PM,1,0.44,2,N,141,162,1,0.0,0.5,13.2,0.00,0.3,66.00,52.0
1,109345316,1,12/25/2017 2:01:07 PM,12/25/2017 2:30:27 PM,2,16.50,2,N,164,132,1,0.0,0.5,8.0,5.76,0.3,66.56,52.0
2,52390640,1,06/20/2017 2:36:54 PM,06/20/2017 3:30:59 PM,1,16.10,2,N,233,132,1,0.0,0.5,11.7,5.76,0.3,70.26,52.0


In [14]:
# prompt: show the rows in Xy_train_2 where fare_amount != 52.0

Xy_train_2[Xy_train_2['fare_amount'] != 52.0]


,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,fare_amount


Rule 2:

The datetime columns need to be of datetime type. For test date, since we use "tpep_pickup_datetime" for reservation datetime, this column would need to be of datetime type.

For train data we are dealing with here, we need to change type for both pickup and dropoff datetime columns.

In [15]:
# Convert `tpep_pickup_datetime` to datetime format
Xy_train_1['tpep_pickup_datetime'] = pd.to_datetime(Xy_train_1['tpep_pickup_datetime'], format='%m/%d/%Y %I:%M:%S %p')

# Convert `tpep_dropoff_datetime` to datetime format
Xy_train_1['tpep_dropoff_datetime'] = pd.to_datetime(Xy_train_1['tpep_dropoff_datetime'], format='%m/%d/%Y %I:%M:%S %p')

In [16]:
# Create `duration` column
Xy_train_1['duration'] = (Xy_train_1['tpep_dropoff_datetime'] - Xy_train_1['tpep_pickup_datetime'])/np.timedelta64(1,'m')

Rule 3: The fare_amount cannot be less than 0.

In [17]:
# Impute values less than $0 with 0
Xy_train_1.loc[Xy_train_1['fare_amount'] < 0, 'fare_amount'] = 0

In [18]:
def compute_upper_threshold(df, col, iqr_factor):
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    upper_threshold = q3 + (iqr_factor * iqr)
    return upper_threshold

In [19]:
fare_am_up_threshold = compute_upper_threshold(Xy_train_1, 'fare_amount', 6)
fare_am_up_threshold

59.0

Rule 4: fare_amount is less or equal to fare_am_up_threshold (59.0). In the training data, all fare_amount that is greater than 59.0, would be imputed to 59.0

This rule is not useful for estimating fare amount. However, it can be useful when comparing the predicted fare_amount with the true amount. For example, we may want to compute performance merices for all rows in test data, and for rows with actual fare_amount <= fare_am_up_threshold.

In [20]:
def outlier_imputer(df, col, upper_threshold):
    # Reassign values > threshold to threshold
    df.loc[df[col] > upper_threshold, col] = upper_threshold
    print(df[col].describe())
    print()

In [21]:
outlier_imputer(Xy_train_1, 'fare_amount', fare_am_up_threshold)

count    17743.000000
mean        11.960391
std          8.730339
min          0.000000
25%          6.500000
50%          9.000000
75%         14.000000
max         59.000000
Name: fare_amount, dtype: float64



In [22]:
# Impute a 0 for any negative values
Xy_train_1.loc[Xy_train_1['duration'] < 0, 'duration'] = 0

Rule 5: 'duration' >= 0. This rule does not apply to real-cases, since so-called 'duration' is not available when user is using the app to get an estimate for the fare amount.

In [23]:
duration_threshold = compute_upper_threshold(Xy_train_1, 'duration', 6)
duration_threshold

84.30833333333331

Rule 6: 'duration' <= duration_threshold. In training data, all "duration" that is greater than the threshold, will be replaced with the threshold itself.

This rule does not apply to real-cases, since so-called 'duration' is not available when user is using the app to get an estimate for the fare amount.

In [24]:
outlier_imputer(Xy_train_1, 'duration', duration_threshold)

count    17743.000000
mean        13.697476
std         10.594089
min          0.000000
25%          6.550000
50%         10.950000
75%         17.658333
max         84.308333
Name: duration, dtype: float64



In [25]:
# Create `pickup_dropoff` column
Xy_train_1['pickup_dropoff'] = Xy_train_1['PULocationID'].astype(str) + ' ' + Xy_train_1['DOLocationID'].astype(str)

Rule 7: Create a concatenated column "pickup_dropoff" with PULocationID and DOLocationID. The column will be used to map into two values, one is related to distance and one is related to duration.

In [26]:
def create_dict_pu_do_to_distance(df):
    grouped = df.groupby('pickup_dropoff').mean(numeric_only=True)[['trip_distance']]
    grouped_dict = grouped.to_dict()
    grouped_dict = grouped_dict['trip_distance']
    return grouped_dict

In [27]:
def create_dict_pu_do_to_duration(df):
    grouped = df.groupby('pickup_dropoff').mean(numeric_only=True)[['duration']]
    grouped_dict = grouped.to_dict()
    grouped_dict = grouped_dict['duration']
    return grouped_dict

In [28]:
def apply_dict(df, new_col, dict_name):
    df[new_col] = df['pickup_dropoff']
    df[new_col] = df[new_col].map(dict_name)

In [29]:
dict_pu_do_to_distance = create_dict_pu_do_to_distance(Xy_train_1)
dict_pu_do_to_duration = create_dict_pu_do_to_duration(Xy_train_1)

In [30]:
default_distance = Xy_train_1['trip_distance'].mean()
default_duration = Xy_train_1['duration'].mean()

Dictionaries:

Dictionary 1: apply dict_pu_do_to_distance to get a distance estimate from pickup_dropoff.

Dictionary 2: apply dict_pu_do_to_duration to get a duration estimate from pickup_dropoff.

Note if a dictionary happen to not have a particular pickup_dropoff value, the dictionary would return NaN.

  * when this case happens, we’ll use a pair of default values: default_distance and default_duration

In [31]:
apply_dict(Xy_train_1, 'distance_estimate', dict_pu_do_to_distance)
apply_dict(Xy_train_1, 'duration_estimate', dict_pu_do_to_duration)

In [32]:
Xy_train_1['day'] = Xy_train_1['tpep_pickup_datetime'].dt.day_name().str.lower()
Xy_train_1['rush_hour'] = Xy_train_1['tpep_pickup_datetime'].dt.hour
Xy_train_1["rush_hour"] = Xy_train_1["rush_hour"].astype('int64')
Xy_train_1.loc[Xy_train_1['day'].isin(['saturday', 'sunday']), 'rush_hour'] = 0

def rush_hourizer(row):
    if 6 <= row['rush_hour'] < 10:
        val = 1
    elif 16 <= row['rush_hour'] < 20:
        val = 1
    else:
        val = 0
    return val

Xy_train_1.loc[(Xy_train_1.day != 'saturday') & (Xy_train_1.day != 'sunday'), 'rush_hour'] \
    = Xy_train_1.apply(rush_hourizer, axis=1)

Computation 1: Compute a rush_hour column based on the pickup datetime (or reservation datetime) in the four steps:

* create a column "day" based on the reservation datetime
* create a column "rush_hour" filled the hour field of the reservation datetime
* set rush_hour to zero for saturday and sunday
* for weekdays, set rush_hour to 1 when hour is in \[6am, 10am\] or \[4pm, 8pm\] -- function rush_hourizer()
  * use apply (see above code block)

In [33]:
Xy_train_1.columns

Index(['Unnamed: 0', 'VendorID', 'tpep_pickup_datetime',
       'tpep_dropoff_datetime', 'passenger_count', 'trip_distance',
       'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID',
       'payment_type', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'fare_amount', 'duration',
       'pickup_dropoff', 'distance_estimate', 'duration_estimate', 'day',
       'rush_hour'],
      dtype='object')

In [34]:
Xy_train_1 = Xy_train_1.drop(['Unnamed: 0', 'tpep_dropoff_datetime', 'tpep_pickup_datetime', 'trip_distance',
               'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID',
               'payment_type', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
               'duration', 'pickup_dropoff',
               'total_amount', 'day'
               ], axis=1)

In [35]:
Xy_train_1.columns

Index(['VendorID', 'passenger_count', 'fare_amount', 'distance_estimate',
       'duration_estimate', 'rush_hour'],
      dtype='object')

In [36]:
X_train_1 = Xy_train_1.drop(['fare_amount'], axis=1)
y_train_1 = Xy_train_1[['fare_amount']]

In [37]:
# Convert VendorID to string
X_train_1['VendorID'] = X_train_1['VendorID'].astype(str)

# Get dummies
X_train_1 = pd.get_dummies(X_train_1, drop_first=True)

In [38]:
X_train_1.columns

Index(['passenger_count', 'distance_estimate', 'duration_estimate',
       'rush_hour', 'VendorID_2'],
      dtype='object')

Rule 8: The model uses five columns to compute the estimated fare amount.
  * 'VendorID_2', (one-hot encoded to "2" = true and "1" = false)
  * 'passenger_count',
  * 'distance_estimate', (created with dict_pu_do_to_distance)
  * 'duration_estimate', (cretaed with dict_pudo_to_duration)
  * 'rush_hour'

In [39]:
scaler_X = StandardScaler().fit(X_train_1)

Computation 2:

use scaler_X to scale each of the five variables as listed in Rule 8. For example, X_train_1_scaled = scaler_X.transform(X_train_1)

In [40]:
X_train_1_scaled = scaler_X.transform(X_train_1)
lr=LinearRegression()
lr.fit(X_train_1_scaled, y_train_1)

LinearRegression()

In [41]:
# Get the coefficients of the linear regression model
coefficients = lr.coef_

# Get the names of the features
feature_names = X_train_1.columns

# Print the coefficients with their corresponding feature names
for feature, coef in zip(feature_names, coefficients[0]):
    print(f"{feature}: {coef:.2f}")

passenger_count: 0.02
distance_estimate: 5.64
duration_estimate: 2.67
rush_hour: 0.14
VendorID_2: -0.07


In [42]:
print(lr.intercept_)

[11.96039114]


Computation 3: Use lr.predict() to compute predictions. For example, y_pred_train_1 = lr.predict(X_train_1_scaled), where X_train_1_scaled is computed with Computation 2.

In [43]:
y_pred_train_1 = lr.predict(X_train_1_scaled)
print('R^2:', r2_score(y_train_1, y_pred_train_1))
print('MAE:', mean_absolute_error(y_train_1, y_pred_train_1))
print('MSE:', mean_squared_error(y_train_1, y_pred_train_1))
print('RMSE:',np.sqrt(mean_squared_error(y_train_1, y_pred_train_1)))

R^2: 0.842473701231825
MAE: 1.9477327983943773
MSE: 12.00579138583178
RMSE: 3.4649374288479984


In [44]:
X_test.columns

Index(['VendorID', 'tpep_pickup_datetime', 'passenger_count', 'RatecodeID',
       'PULocationID', 'DOLocationID'],
      dtype='object')

In [47]:
# prompt: reset index of X_test and y_test

X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)


In [48]:
#this is for Rule 1 - keeping the column of RatecodeID
X_test_RatecodeID = X_test[['RatecodeID']]

In [ ]:
X_test_RatecodeID.head()

In [50]:
#this is for Rule 2 - changing to datetime type
X_test['tpep_pickup_datetime'] = pd.to_datetime(X_test['tpep_pickup_datetime'], format='%m/%d/%Y %I:%M:%S %p')

In [51]:
# prompt: Create a concatenated column "pickup_dropoff" with PULocationID and DOLocationID.

# Create `pickup_dropoff` column
X_test['pickup_dropoff'] = X_test['PULocationID'].astype(str) + ' ' + X_test['DOLocationID'].astype(str)


In [52]:
# prompt: apply dict_pu_do_to_distance to get a column of distance estimate from pickup_dropoff.

apply_dict(X_test, 'distance_estimate', dict_pu_do_to_distance)


In [53]:
# prompt: apply dict_pu_do_to_duration to get a column of duration estimate from pickup_dropoff.

apply_dict(X_test, 'duration_estimate', dict_pu_do_to_duration)


In [54]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4540 entries, 0 to 4539
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   VendorID              4540 non-null   int64         
 1   tpep_pickup_datetime  4540 non-null   datetime64[ns]
 2   passenger_count       4540 non-null   int64         
 3   RatecodeID            4540 non-null   int64         
 4   PULocationID          4540 non-null   int64         
 5   DOLocationID          4540 non-null   int64         
 6   pickup_dropoff        4540 non-null   object        
 7   distance_estimate     4075 non-null   float64       
 8   duration_estimate     4075 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(5), object(1)
memory usage: 319.3+ KB


In [55]:
# prompt: replace NaNs in column duration_estimate with default_duration, and in column distance_estimate with default_distance

X_test['duration_estimate'] = X_test['duration_estimate'].fillna(default_duration)
X_test['distance_estimate'] = X_test['distance_estimate'].fillna(default_distance)


In [56]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4540 entries, 0 to 4539
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   VendorID              4540 non-null   int64         
 1   tpep_pickup_datetime  4540 non-null   datetime64[ns]
 2   passenger_count       4540 non-null   int64         
 3   RatecodeID            4540 non-null   int64         
 4   PULocationID          4540 non-null   int64         
 5   DOLocationID          4540 non-null   int64         
 6   pickup_dropoff        4540 non-null   object        
 7   distance_estimate     4540 non-null   float64       
 8   duration_estimate     4540 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(5), object(1)
memory usage: 319.3+ KB


In [57]:
# prompt:     * create a column "day" based on the reservation datetime
#     * create a column "rush_hour" filled the hour field of the reservation datetime
#     * set rush_hour to zero for saturday and sunday
#     * for weekdays, set rush_hour to 1 when hour is in [6am, 10am] or [4pm, 8pm] by applying function rush_hourizer()

X_test['day'] = X_test['tpep_pickup_datetime'].dt.day_name().str.lower()
X_test['rush_hour'] = X_test['tpep_pickup_datetime'].dt.hour
X_test["rush_hour"] = X_test["rush_hour"].astype('int64')
X_test.loc[X_test['day'].isin(['saturday', 'sunday']), 'rush_hour'] = 0

X_test.loc[(X_test.day != 'saturday') & (X_test.day != 'sunday'), 'rush_hour'] \
    = X_test.apply(rush_hourizer, axis=1)


In [58]:
# prompt: drop the following columns from X_test: tpep_pickup_datetime, RatecodeID, PULocationID, DOLocationID, pickup_dropoff.

X_test = X_test.drop(['tpep_pickup_datetime', 'RatecodeID', 'PULocationID', 'DOLocationID', 'pickup_dropoff'], axis=1)


In [60]:
X_test = X_test.drop(['day'], axis=1)

In [61]:
X_test.columns

Index(['VendorID', 'passenger_count', 'distance_estimate', 'duration_estimate',
       'rush_hour'],
      dtype='object')

In [62]:
# Convert VendorID to string
X_test['VendorID'] = X_test['VendorID'].astype(str)

# Get dummies
X_test = pd.get_dummies(X_test, drop_first=True)

In [63]:
X_test.columns

Index(['passenger_count', 'distance_estimate', 'duration_estimate',
       'rush_hour', 'VendorID_2'],
      dtype='object')

In [64]:
# prompt: apply scaler_X on X_test

# Scale the test data using the scaler fitted on the training data
X_test_scaled = scaler_X.transform(X_test)


In [65]:
# prompt: use model lr to compute y_test_pred with X_test_scaled

# Predict on the scaled test data
y_test_pred = lr.predict(X_test_scaled)


In [66]:
# prompt: replace the values in y_test_pred with 52.0 for the rows where the corresponding RatecodeID == 2 in X_test_RatecodeID

# Create a boolean mask for rows where RatecodeID == 2
mask = X_test_RatecodeID['RatecodeID'] == 2

# Replace values in y_test_pred with 52.0 for rows where mask is True
y_test_pred[mask] = 52.0


In [67]:
# prompt: compute the following metrices between y_test and y_test_pred: r2_score, mean absolute error, mean squared error, and square-root of mean squared error

# Calculate metrics
print('R^2:', r2_score(y_test, y_test_pred))
print('MAE:', mean_absolute_error(y_test, y_test_pred))
print('MSE:', mean_squared_error(y_test, y_test_pred))
print('RMSE:',np.sqrt(mean_squared_error(y_test, y_test_pred)))


R^2: 0.6619173034758208
MAE: 3.067743872970842
MSE: 39.79685412831389
RMSE: 6.308474786215277


In [ ]:
# prompt: display the rows in y_test where the fare_amount < 0, or > fare_am_up_threshold

# Filter the dataframe
result = y_test[(y_test['fare_amount'] < 0) | (y_test['fare_amount'] > fare_am_up_threshold)]

# Display the result
display(result)


In [69]:
# prompt: compute the following metrics between y_test and y_test_pred, excluding the values where fare_amount in y_test < 0 or > fare_am_up_threshold: r2_score, mean absolute error, mean squared error, and square-root of mean squared error

# Create a boolean mask for valid fare amounts
valid_fares = (y_test['fare_amount'] >= 0) & (y_test['fare_amount'] <= fare_am_up_threshold)

# Filter y_test and y_test_pred based on valid fares
y_test_filtered = y_test[valid_fares]['fare_amount']
y_test_pred_filtered = y_test_pred[valid_fares]

# Calculate metrics on filtered data
print('R^2:', r2_score(y_test_filtered, y_test_pred_filtered))
print('MAE:', mean_absolute_error(y_test_filtered, y_test_pred_filtered))
print('MSE:', mean_squared_error(y_test_filtered, y_test_pred_filtered))
print('RMSE:', np.sqrt(mean_squared_error(y_test_filtered, y_test_pred_filtered)))


R^2: 0.7380743723892089
MAE: 2.865137065295924
MSE: 25.88382796407055
RMSE: 5.087615154870752


In [70]:
X_test.columns

Index(['passenger_count', 'distance_estimate', 'duration_estimate',
       'rush_hour', 'VendorID_2'],
      dtype='object')

In [72]:
y_test_pred


array([[11.77285971],
       [15.09162535],
       [ 6.55921487],
       ...,
       [ 6.71559236],
       [ 6.3615011 ],
       [11.4474799 ]])

In [73]:
# prompt: create a dataframe called test_estimates_preds with three columns: the first column is the column duration_estimate in X_test, the second column is the column distance_estimate in X_test, and the third column in y_test_pred.

test_estimates_preds = pd.DataFrame({
    'duration_estimate': X_test['duration_estimate'],
    'distance_estimate': X_test['distance_estimate'],
    'y_test_pred': y_test_pred.flatten()  # Flatten the predictions array
})


In [74]:
test_estimates_preds.head()

,duration_estimate,distance_estimate,y_test_pred
0,12.141667,2.812500,11.772860
1,19.138889,3.410000,15.091625
2,7.550000,0.901429,6.559215
3,15.509091,4.478182,16.136568
4,12.750000,1.955000,10.389788


In [76]:
X_train_1.columns

Index(['passenger_count', 'distance_estimate', 'duration_estimate',
       'rush_hour', 'VendorID_2'],
      dtype='object')

In [77]:
y_pred_train_1

array([[19.44601409],
       [ 6.73799512],
       [ 7.65893996],
       ...,
       [ 7.71721609],
       [ 6.20628968],
       [21.63469497]])

In [78]:
X_train_1.head()

,passenger_count,distance_estimate,duration_estimate,rush_hour,VendorID_2
0,1,6.020000,16.216667,0,True
1,1,0.807273,7.781818,1,True
2,2,1.184615,8.839744,0,False
3,1,0.854444,5.762963,1,False
4,1,0.898125,6.412500,1,False


In [79]:
# prompt: create a dataframe called train_1_estimates_preds with three columns: the first column is the column duration_estimate in X_train_1, the second column is the column distance_estimate in X_train_1, and the third column in y_pred_train_1.

train_1_estimates_preds = pd.DataFrame({
    'duration_estimate': X_train_1['duration_estimate'],
    'distance_estimate': X_train_1['distance_estimate'],
    'y_pred_train_1': y_pred_train_1.flatten()  # Flatten the predictions array
})

train_1_estimates_preds.head()


,duration_estimate,distance_estimate,y_pred_train_1
0,16.216667,6.020000,19.446014
1,7.781818,0.807273,6.737995
2,8.839744,1.184615,7.658940
3,5.762963,0.854444,6.361826
4,6.412500,0.898125,6.646244


In [80]:
Xy_train_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416 entries, 0 to 415
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             416 non-null    int64  
 1   VendorID               416 non-null    int64  
 2   tpep_pickup_datetime   416 non-null    object 
 3   tpep_dropoff_datetime  416 non-null    object 
 4   passenger_count        416 non-null    int64  
 5   trip_distance          416 non-null    float64
 6   RatecodeID             416 non-null    int64  
 7   store_and_fwd_flag     416 non-null    object 
 8   PULocationID           416 non-null    int64  
 9   DOLocationID           416 non-null    int64  
 10  payment_type           416 non-null    int64  
 11  extra                  416 non-null    float64
 12  mta_tax                416 non-null    float64
 13  tip_amount             416 non-null    float64
 14  tolls_amount           416 non-null    float64
 15  improv

In [81]:
# prompt: create a dataframe called train_2_estimates_preds with the same number of rows as Xy_train_2, and with three columns: the first column is named as duration_estimate with the same value of default_duration,  the second column is named as distance_estimate with the same value of default_distance,  and the third column is called fare_amount with 52.0 .

# Create a DataFrame with default values
train_2_estimates_preds = pd.DataFrame({
    'duration_estimate': [default_duration] * len(Xy_train_2),
    'distance_estimate': [default_distance] * len(Xy_train_2),
    'fare_amount': [52.0] * len(Xy_train_2)
})

# Display the DataFrame
train_2_estimates_preds.head()


,duration_estimate,distance_estimate,fare_amount
0,13.697476,2.589797,52.0
1,13.697476,2.589797,52.0
2,13.697476,2.589797,52.0
3,13.697476,2.589797,52.0
4,13.697476,2.589797,52.0


In [84]:
# prompt: rename the column names for the following three data frames: est_estimates_preds, train_1_estimates_preds, and train_2_estimates_preds. The first column is to be renamed as "a", the second column to "b", and the third column to "c".

# Rename columns for test_estimates_preds
test_estimates_preds = test_estimates_preds.rename(columns={
    'duration_estimate': 'a',
    'distance_estimate': 'b',
    'y_test_pred': 'c'
})

# Rename columns for train_1_estimates_preds
train_1_estimates_preds = train_1_estimates_preds.rename(columns={
    'duration_estimate': 'a',
    'distance_estimate': 'b',
    'y_pred_train_1': 'c'
})

# Rename columns for train_2_estimates_preds
train_2_estimates_preds = train_2_estimates_preds.rename(columns={
    'duration_estimate': 'a',
    'distance_estimate': 'b',
    'fare_amount': 'c'
})


In [85]:
# prompt: concatenate test_estimates_preds, train_1_estimates_preds, and train_2_estimates_preds together vertically, and call the columns as "mean_duration", "mean_distance", and "predicted_fare" respectively. Reset index. Call the resulting dataframe as nyc_preds_means

# Concatenate the DataFrames vertically
nyc_preds_means = pd.concat([test_estimates_preds, train_1_estimates_preds, train_2_estimates_preds], axis=0)

# Rename columns
nyc_preds_means = nyc_preds_means.rename(columns={
    'a': 'mean_duration',
    'b': 'mean_distance',
    'c': 'predicted_fare'
})

# Reset index
nyc_preds_means = nyc_preds_means.reset_index(drop=True)

# Display the result
nyc_preds_means.head()


,mean_duration,mean_distance,predicted_fare
0,12.141667,2.812500,11.772860
1,19.138889,3.410000,15.091625
2,7.550000,0.901429,6.559215
3,15.509091,4.478182,16.136568
4,12.750000,1.955000,10.389788


In [86]:
# prompt: display the the shape info for test_estimates_preds, train_1_estimates_preds, train_2_estimates_preds, and nyc_preds_means. Verify that the number of rows for nyc_preds_means equals to the sum of numbers of rows for test_estimates_preds, train_1_estimates_preds, and train_2_estimates_preds.

# Display shape information
print("Shape of test_estimates_preds:", test_estimates_preds.shape)
print("Shape of train_1_estimates_preds:", train_1_estimates_preds.shape)
print("Shape of train_2_estimates_preds:", train_2_estimates_preds.shape)
print("Shape of nyc_preds_means:", nyc_preds_means.shape)

# Verify the number of rows
total_rows = test_estimates_preds.shape[0] + train_1_estimates_preds.shape[0] + train_2_estimates_preds.shape[0]
print("Total rows from individual DataFrames:", total_rows)

if total_rows == nyc_preds_means.shape[0]:
    print("Verification successful: Number of rows in nyc_preds_means matches the sum of individual DataFrames.")
else:
    print("Verification failed: Number of rows mismatch.")


Shape of test_estimates_preds: (4540, 3)
Shape of train_1_estimates_preds: (17743, 3)
Shape of train_2_estimates_preds: (416, 3)
Shape of nyc_preds_means: (22699, 3)
Total rows from individual DataFrames: 22699
Verification successful: Number of rows in nyc_preds_means matches the sum of individual DataFrames.


### 2. Impute ratecode 2 fare

The data dictionary says that the `RatecodeID` column captures the following information:

1 = standard rate  
2 = JFK (airport)  
3 = Newark (airport)  
4 = Nassau or Westchester  
5 = Negotiated fare  
6 = Group ride  

This means that some fares don't need to be predicted. They can simply be imputed based on their rate code. Specifically, all rate codes of `2` can be imputed with \$52, as this is a flat rate for JFK airport.

The other rate codes have some variation (not shown here, but feel free to check for yourself). They are not a fixed rate, so these fares will remain untouched.

Impute `52` at all predictions where `RatecodeID` is `2`.

In [ ]:
# Create a new df containing just the RatecodeID col from the whole dataset
final_preds = df[['RatecodeID']].copy()

# Add a column containing all the predictions
final_preds['y_preds_full'] = y_preds_full

# Impute a prediction of 52 at all rows where RatecodeID == 2
final_preds.loc[final_preds['RatecodeID']==2, 'y_preds_full'] = 52

# Check that it worked
final_preds[final_preds['RatecodeID']==2].head()

,RatecodeID,y_preds_full
11,2,52.0
110,2,52.0
161,2,52.0
247,2,52.0
379,2,52.0


### Check performance on full dataset

In [ ]:
final_preds = final_preds['y_preds_full']
print('R^2:', r2_score(y, final_preds))
print('MAE:', mean_absolute_error(y, final_preds))
print('MSE:', mean_squared_error(y, final_preds))
print('RMSE:',np.sqrt(mean_squared_error(y, final_preds)))

R^2: 0.8910853978683975
MAE: 1.992506252269974
MSE: 12.101575504689935
RMSE: 3.4787318816905013


### Save final predictions with `mean_duration` and `mean_distance` columns

In [ ]:
# Combine means columns with predictions column
nyc_preds_means = df[['mean_duration', 'mean_distance']].copy()
nyc_preds_means['predicted_fare'] = final_preds

nyc_preds_means.head()

,mean_duration,mean_distance,predicted_fare
0,22.847222,3.521667,16.434245
1,24.470370,3.108889,16.052218
2,7.250000,0.881429,7.053706
3,30.250000,3.700000,18.731650
4,14.616667,4.435000,15.845642


Save as a csv file

# NOTES

This notebook was designed for teaching purposes. As such, there are some things to note that differ from best practice or from how tasks are typically performed.

1.  When the `mean_distance` and `mean_duration` columns were computed, the means were calculated from the entire dataset. These same columns were then used to train a model that was used to predict on a test set. A test set is supposed to represent entirely new data that the model has not seen before, but in this case, some of its predictor variables were derived using data that *was* in the test set.</br></br>
This is known as **<u>data leakage</u>**. Data leakage is when information from your training data contaminates the test data. If your model has unexpectedly high scores, there is a good chance that there was some data leakage.
</br></br>
To avoid data leakage in this modeling process, it would be best to compute the means using only the training set and then copy those into the test set, thus preventing values from the test set from being included in the computation of the means. This would have created some problems because it's very likely that some combinations of pickup-dropoff locations would only appear in the test data (not the train data). This means that there would be NaNs in the test data, and further steps would be required to address this.
</br></br>
In this case, the data leakage improved the R<sup>2</sup> score by ~0.03.
</br></br>
2. Imputing the fare amount for `RatecodeID 2` after training the model and then calculating model performance metrics on the post-imputed data is not best practice. It would be better to separate the rides that did *not* have rate codes of 2, train the model on that data specifically, and then add the `RatecodeID 2` data (and its imputed rates) *after*. This would prevent training the model on data that you don't need a model for, and would likely result in a better final model. However, the steps were combined for simplicity.
</br></br>
3. Models that predict values to be used in another downstream model are common in data science workflows. When models are deployed, the data cleaning, imputations, splits, predictions, etc. are done using modeling pipelines. Pandas was used here to granularize and explain the concepts of certain steps, but this process would be streamlined by machine learning engineers. The ideas are the same, but the implementation would differ. Once a modeling workflow has been validated, the entire process can be automated, often with no need for pandas and no need to examine outputs at each step. This entire process would be reduced to a page of code.